In [1]:
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *
from utils import *

import torch.nn as nn
import torch.nn.functional as F
import torch

from dataset.time_dataset import TsDataset
from pathlib import Path

In [2]:
# Dataset loader
datapath = Path('data')
dataset = TsDataset(datapath/'CaseI-Attacks without any change.csv')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                        shuffle=True, num_workers=4)
print("The size of the dataset is: ", dataset.data_normal.size(), dataset.data_abnormal.size())

The size of the dataset is:  torch.Size([33, 576]) torch.Size([33, 15, 576])


In [3]:
normal_dim = dataset.data_normal.size(1)
abnormal_dim = dataset.data_abnormal.size(2)
in_dim = dataset.__len__()

In [4]:
G_AB = LSTMGenerator(normal_dim, normal_dim)
G_BA = LSTMGenerator(abnormal_dim, abnormal_dim)
D_A = LSTMDiscriminator(normal_dim)
D_B = LSTMDiscriminator(abnormal_dim)

In [5]:
G_AB.load_state_dict(torch.load("saved_models/cyber/G_AB_190.pth"))
G_BA.load_state_dict(torch.load("saved_models/cyber/G_BA_190.pth"))
D_A.load_state_dict(torch.load("saved_models/cyber/D_A_190.pth"))
D_B.load_state_dict(torch.load("saved_models/cyber/D_B_190.pth"))

<All keys matched successfully>

In [6]:
G_AB.eval()
G_BA.eval()
D_A.eval()
D_B.eval()

LSTMDiscriminator(
  (lstm): LSTM(576, 256, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [7]:
dataset.data_normal.size()

torch.Size([33, 576])

In [8]:
with torch.no_grad():
    output = G_AB(dataset.data_normal.unsqueeze(1))
    output_back = G_BA(output)

In [9]:
output.size(), output_back.size()

(torch.Size([33, 1, 576]), torch.Size([33, 1, 576]))

In [10]:
import matplotlib.pyplot as plt
for i in range(output.size(0)):
    plt.plot(output[i, 0, :].numpy())
    plt.plot(output_back[i, 0, :].numpy())
    plt.plot(dataset.data_normal[i, :].numpy())
    plt.legend(['Generated', 'Generated Back', 'Original'])
    plt.savefig(f'cyber_result/forward/output_{i}.png')
    plt.close()

---

In [11]:
dataset.data_abnormal.size()

torch.Size([33, 15, 576])

In [12]:
data_abnormal_test = dataset.data_abnormal[:,0,:]
data_abnormal_test.size()

torch.Size([33, 576])

In [13]:
with torch.no_grad():
    output = G_AB(data_abnormal_test.unsqueeze(1))
    output_back = G_BA(output)

In [14]:
for i in range(output.size(0)):
    plt.plot(output[i, 0, :].numpy())
    plt.plot(output_back[i, 0, :].numpy())
    plt.plot(data_abnormal_test[i, :].numpy())
    plt.legend(['Generated', 'Generated Back', 'Original'])
    plt.savefig(f'cyber_result/detection/output_{i}.png')
    plt.close()